In [64]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
# Importamos las librerías necesarias
import numpy as np
import time
import cv2 as cv
import matplotlib.pyplot as plt
import os
from pygame import mixer
#import pygame

In [65]:
#--------------------------------------------------------------------------------
#face_cascade = cv.CascadeClassifier(facepath)
eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_eye.xml")
face_cascade=cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml") #Note the change

In [66]:
# Ratio binariza la imagen y calcula la proporcion entre pixeles blancos y negros

def ratio(eye1,eye2):

    ret1,thresh1 = cv.threshold(eye1,127,255,cv.THRESH_BINARY)
    ret2,thresh2 = cv.threshold(eye2,127,255,cv.THRESH_BINARY)
    number_of_white_pix = np.sum(thresh1 == 255)     
    number_of_black_pix = np.sum(thresh1 == 0)         
    ratio_white_pix_1=number_of_white_pix/number_of_black_pix
    number_of_white_pix = np.sum(thresh2 == 255)     
    number_of_black_pix = np.sum(thresh2 == 0)          
    ratio_white_pix_2=number_of_white_pix/number_of_black_pix
    ratio_white_pix=(ratio_white_pix_2+ratio_white_pix_1)/2
    return ratio_white_pix

In [68]:
# Condición de terminación: 10 iteraciones o moverse al menos 1pt
#----------------------------------------------------------------
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )
cap = cv.VideoCapture(0)
path=r"C:\Users\54922\GITHUB\vision_computadora_I\TP_Entrega_vision_por_computadora_1\Tp6.avi"
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv.VideoWriter(path,cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))


k=0
frame_count=0

param = 0
metricas = 0
metricasP = 0
primera_lectura = True
contador = 0
contador2 = 0
n = 5

counter = 0
time = 0
thick = 2



while(1):

    ret ,frame = cap.read()

    if ret == True:
        key = False
        

        if key == True:
            break
        else:
            originalImage = frame
            # Imagen a escala de grises
            grayImage = cv.cvtColor(originalImage, cv.COLOR_BGR2GRAY)
            grayImage = cv.equalizeHist(grayImage)

            #Vamos a hacer una sola detección satisfactoria y despues usaremos el mismo roi fijo para todos los frame
            
            # Llamamos al clasificador de Haar (AdaBoost)

            faces = face_cascade.detectMultiScale(grayImage,minNeighbors=20,scaleFactor=1.1,minSize=(25,25))# gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25)
        

            # Recorro las caras encontradas
            #------------------------------
            for (x,y,w,h) in faces:
                cv.putText(originalImage, 'Presione \"q\" para salir',(80,450), cv.FONT_HERSHEY_PLAIN, 1.5,(0,255,255),2)
                # Le dibujamos un rectángulo rosa
                cv.rectangle(originalImage,(x,y),(x+w,y+h),(255,0,0),2)
                # Definimos las ROIs en la imagen gris y color
                
                roi_gray = grayImage[y:y+h, x:x+w] 
                roi_color = originalImage[y:y+h, x:x+w] 
                #Pasamos roi a hsv
                hsv_roi =  cv.cvtColor(roi_color, cv.COLOR_BGR2HSV)


                # Para cada rostro hallado le buscamos los ojos
                eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 8,minSize= (8,13),maxSize=(200,400))
                

                # En los ojos hallados les dibujamos rectángulos
                for (ex,ey,ew,eh) in eyes:
                    cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)
                    cv.putText(originalImage, str(len(eyes)),(10,30), cv.FONT_HERSHEY_PLAIN, 1,(0,0,0),2)


                # Detecto si el programa está viendo los ojos
                if (primera_lectura) and (len(eyes)==2):
                    cv.putText(originalImage, 'Ojos detectados, \"s\" para empezar',(20,80), cv.FONT_HERSHEY_PLAIN, 2,(255,255,0),2)

                elif primera_lectura == False:
                    # ubico el ojo en la posición relativa comparando con el primer frame
                    ojo1 = roi_gray[ojo_rel1_y:(ojo_rel1_y+ojo_rel1_h), ojo_rel1_x:(ojo_rel1_x+ojo_rel1_w)]
                    ojo2 = roi_gray[ojo_rel2_y:(ojo_rel2_y+ojo_rel2_h), ojo_rel2_x:(ojo_rel2_x+ojo_rel2_w)]
                


                
                    metrica_ratio= ratio(ojo1_f,ojo2_f)/ratio(ojo1,ojo2)
                    #print(metrica_ratio)
                    cv.putText(originalImage, 'param.: ' + str(metrica_ratio),(10,140), cv.FONT_HERSHEY_PLAIN, 2,(0,255,0),2)
                    if metrica_ratio < 0.50:
                        frame_count=frame_count+1
                        print("Count",frame_count,"metrica",metrica_ratio)
                    if frame_count>=4: # and metrica_ratio < 0.30:
                        print("Ingresa al loop, pero no imprime cartel ni lanza alerta")
                        frame_count=0
                        cv.putText(frame,'Wake up Time !!:'+ str(frame_count),(10,3000), cv.FONT_HERSHEY_PLAIN, 3,(0,0,255),2,cv.LINE_AA)
                        try:
                            alarm_sound.play()
        
                        except:  
                            pass

                    

            cv.imshow('Deteccion de ojos cerrados',originalImage)
            w = cv.waitKey(1)
            if(w==ord('q')):
                break
            elif(w==ord('s') and primera_lectura):
                #This will start the detection
                if(len(eyes)==2):
                    primera_lectura = False

                    ojo_rel1_x = eyes[0][0]
                    ojo_rel1_y = eyes[0][1]
                    ojo_rel1_w = eyes[0][2]
                    ojo_rel1_h = eyes[0][3]
                    ojo1_f = roi_gray[ey:ey+eh, ex:ex+ew]
                  #  centro_ojo1_f = hist_center(ojo1_f)

                    ojo_rel2_x = eyes[1][0]
                    ojo_rel2_y = eyes[1][1]
                    ojo_rel2_w = eyes[1][2]
                    ojo_rel2_h = eyes[1][3] 
                    ojo2_f = roi_gray[ey:ey+eh, ex:ex+ew]
                   # centro_ojo2_f = hist_center(ojo1_f)
                
            out.write(frame)

    else: 
        break


cap.release()
out.release()
cv.destroyAllWindows()

Count 1 metrica 0.30103732739436284
Count 2 metrica 0.3125407241993586
Count 3 metrica 0.29106041590353626
Count 4 metrica 0.3118398370628248
Ingresa al loop, pero no imprime cartel ni lanza alerta
Count 1 metrica 0.31256647323527537
Count 2 metrica 0.23022877095170705
Count 3 metrica 0.2911666338776313
Count 4 metrica 0.22009746261132582
Ingresa al loop, pero no imprime cartel ni lanza alerta
Count 1 metrica 0.19964271605604214
Count 2 metrica 0.26073713027991346
Count 3 metrica 0.23106176433853454
Count 4 metrica 0.2098986929975802
Ingresa al loop, pero no imprime cartel ni lanza alerta
Count 1 metrica 0.2816840189415994
Count 2 metrica 0.31085349099138365
Count 3 metrica 0.2817286890702188
Count 4 metrica 0.29263676043197095
Ingresa al loop, pero no imprime cartel ni lanza alerta
Count 1 metrica 0.2511126957975754
Count 2 metrica 0.3015417473901787
Count 3 metrica 0.331913802961298
Count 4 metrica 0.2822622098968084
Ingresa al loop, pero no imprime cartel ni lanza alerta
Count 1 met